In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from src.networks.cnn_model import CNNModel
from src.networks.fc_model import FCModel
from src.util.test_model import test_model

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'



ModuleNotFoundError: No module named 'src'

In [2]:
import numpy as np

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
import torchvision.transforms as transforms

%load_ext autoreload
%autoreload 2


In [3]:
# Prepare train and test data
batch_size = 4

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
hparams = {"batch_size": 16 ,
          "learning_rate": 0.001,
          "num_filters": 32,
          "kernel_size": 3,
          "stride": 1,
          "padding": 1,
          "n_hidden": 128}

In [5]:
model = FCModel(hparams=hparams)

train = model.train_dataloader()
epochs = 10

optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9, dampening=2e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

for epoch in range(epochs):
    train_losses = []
    for img in train:
    
        optimizer.zero_grad()
        outputs = model(inputs) 

        # inputs [10, 3, 240, 240]
        # targets [240, 240]
        # outputs [10, 23, 240, 240]

        loss = 0
        loss = criterion(outputs, targets)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    targets_mask = targets >= 0
    train_acc = np.mean((preds == targets)[targets_mask].data.cpu().numpy())
    print('[Epoch %d/%d] TRAIN acc/loss: %.3f/%.3f' % (epoch + 1,
                                                        epochs,
                                                        train_acc,
                                                        train_losses[-1]))


NameError: name 'FCModel' is not defined

In [6]:
model = FCModel(hparams)
model.prepare_data()

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)

trainer = pl.Trainer(max_epochs=3)
trainer.fit(model)

NameError: name 'FCModel' is not defined

In [7]:
model = classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
epochs = 10

print('Start training!')
for e in range(epochs):
    print('Start training epoch', e+1)
    train_loss = 0
    
    for data, target in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)	#loss.item()是平均损失，平均损失*batch_size=一次训练的损失
        
    train_loss = train_loss/len(trainloader.dataset)
    
    print('Epoch: {} \t Training Loss:{:.6f}'.format(e+1, train_loss))

NameError: name 'classifier' is not defined

In [8]:
def test_accuracy():
    classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print('Total Accuracy: %2d %%' % (100 * sum(class_correct) / sum(class_total)))

    for i in range(10): 
        print('Accuracy of %5s : %2f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [26]:
test_accuracy()

Total Accuracy: 68 %
Accuracy of plane : 79.600000 %
Accuracy of   car : 80.400000 %
Accuracy of  bird : 58.600000 %
Accuracy of   cat : 50.700000 %
Accuracy of  deer : 62.700000 %
Accuracy of   dog : 56.700000 %
Accuracy of  frog : 75.300000 %
Accuracy of horse : 67.700000 %
Accuracy of  ship : 78.200000 %
Accuracy of truck : 77.300000 %


In [33]:
model = CNNModel(hparams)
model.prepare_data()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
epochs = 10
train_loader = model.train_dataloader()

for e in range(epochs):
    train_loss = 0
    
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    train_loss = train_loss/len(train_loader)
    
    print('Epoch: {} \t Training Loss:{:.6f}'.format(e+1, train_loss))

torch.Size([16, 128, 4, 4])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1